In [ ]:
import pandas as pd

In [ ]:
trade_kwargs = {'index_col': 0, 'parse_dates': True}
pnl_kwargs = {'index_col': 0, 'parse_dates': True, 'usecols': [0, 1, 2, 4, 5, 6, 10, 11, 12, 13]}
trade_cont = pd.read_csv('result/RB888.SHFE_trade_continuous.csv', **trade_kwargs)
trade_seg = pd.read_csv('result/rb_trade_seg.csv', **trade_kwargs)
pnl_cont = pd.read_csv('result/RB888.SHFE_pnl_continuous.csv', **pnl_kwargs)
pnl_seg = pd.read_csv('result/rb_pnl_seg.csv', **pnl_kwargs)

In [ ]:
trade_cont.head()
# trade_cont.info()
pnl_cont.head()

In [ ]:
trade_seg.head()
pnl_seg.head()

In [ ]:
trade_comp = trade_cont.merge(trade_seg, left_index=True, right_index=True, how='outer', suffixes=('_cont','_seg'))
pnl_comp = pnl_cont.merge(pnl_seg, left_index=True, right_index=True, how='outer', suffixes=('_cont','_seg'))

In [ ]:
# trade_comp.head()
# 选取拼接合约成交点位和分段回测不同的单
df = trade_comp[trade_comp.isnull().T.any()]
# df
num = len(df[df['direction_cont'].notnull()])
num / len(trade_cont)

In [ ]:
pnl_comp.head()
df = pnl_comp[pnl_comp.net_pnl_cont != pnl_comp.net_pnl_seg]


In [ ]:
# 选取成交点位相同但是方向不一致的单
df = trade_comp[~trade_comp.isnull().T.any()]
df[df.direction_cont != df.direction_seg]


In [ ]:
trade_comp.to_csv('result/rb_trade_comp.csv')